In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import dotenv
import os
import sys
import numpy as np
from tqdm import tqdm

In [3]:
# add root project directory
sys.path.append("../")
# get environment path file
dotenv_path = dotenv.find_dotenv()
# load environment variables
dotenv.load_dotenv(dotenv_path)

CALENDAR_FILE_PATH = os.environ.get("CALENDAR_FILE_PATH")
SALES_TRAIN_EVALUATION_FILE_PATH = os.environ.get("SALES_TRAIN_EVALUATION_FILE_PATH")
SALES_TRAIN_VALIDATION_FILE_PATH = os.environ.get("SALES_TRAIN_VALIDATION_FILE_PATH")
SAMPLE_SUBMISSION_FILE_PATH = os.environ.get("SAMPLE_SUBMISSION_FILE_PATH")
SELL_PRICES_FILE_PATH = os.environ.get("SELL_PRICES_FILE_PATH")

In [5]:
# load dataset
calendar = pd.read_csv(CALENDAR_FILE_PATH)
sell_price = pd.read_csv(SELL_PRICES_FILE_PATH)
sales_train_validation = pd.read_csv(SALES_TRAIN_VALIDATION_FILE_PATH)
calendar_ori = calendar.copy()
sell_price_ori = sell_price.copy()
sales_train_validation_ori = sales_train_validation.copy()

In [6]:
display(calendar.head())
display(sell_price.head())
display(sales_train_validation.head())

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


# Naive Baseline Forecast

Evaluate the model
1. Calculate the weight for the 12 level of aggregation
2. Perform naive forecast on the each of the level
3. Infer forecst, ground truth values, and weights for all the higher level series by aggregating
4. Calculate RMSSE for all the series
5. Multiple the weight to each repective RMSSE and sum it all up

In [7]:
calendar['d'] = calendar_ori['d'].apply(lambda x: x.split('_')[1]).astype(int)
sell_price['id'] = sell_price_ori['item_id'] + '_' + sell_price_ori['store_id'] + '_validation'

## 1. Calculate the weight for the 12 level of aggregation

In [8]:
for day in tqdm(range(1858, 1886)):
    wk_id = list(calendar[calendar['d']==day]['wm_yr_wk'])[0]  # select the wm_yr_wk
    wk_price_df = sell_price[sell_price['wm_yr_wk']==wk_id]  # filter based on wm_yr_wk
    sales_train_validation = sales_train_validation.merge(wk_price_df[['id', 'sell_price']], on=['id'], how='inner')  # merge it with the main transaction data
    sales_train_validation['unit_sales_' + str(day)] = sales_train_validation['sell_price'] * sales_train_validation['d_' + str(day)]  # calculate the total sell with the sell price on that specific day
    sales_train_validation.drop(columns=['sell_price'], inplace=True)  # drop the sell price

100%|██████████| 28/28 [00:23<00:00,  1.21it/s]


In [9]:
sales_train_validation['dollar_sales'] = sales_train_validation[[c for c in sales_train_validation.columns if c.find("unit_sales")==0]].sum(axis=1)

In [10]:
sales_train_validation.drop(columns=[c for c in sales_train_validation.columns if c.find("unit_sales")==0], inplace=True)

In [11]:
sales_train_validation['weight'] = sales_train_validation['dollar_sales'] / sales_train_validation['dollar_sales'].sum() 

In [12]:
sales_train_validation.drop(columns=['dollar_sales'], inplace=True)

## 2. Perform naive forecast on the each of the level
There are multiple choices such as:
- add all 0s to the prediction
- average through all the history (**exclude day where the sales is 0**)
- same as previous 28 days
- mean of pervious 10, 20, 30, 40, 50, 60 days
- Average of same day for all previous weeks 

### using all 0's for prediction

In [13]:
# using all 0 for the prediction
for d in range(1886, 1914):
    sales_train_validation['f_0' + str(d)] = 0

method_dict = {0 : "using all 0s"}

### Complete historical average and non zero historical average

In [14]:
# historical average
complete_historical_mean = sales_train_validation[[c for c in sales_train_validation if c.find('d_')==0 and int(c.split('_')[1]) <= 1885]]\
    .mean(axis=1)

In [15]:
# non_zero_historical_mean = 
def calc_non_zero_mean(series):
    assert type(series) == np.ndarray
    return series[series!=0].mean()

non_zero_historical_mean = []
historical_arr = np.array(sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_')==0]])
for i in tqdm(range(len(sales_train_validation))):
    non_zero_historical_mean.append(round(calc_non_zero_mean(historical_arr[i, :])))

100%|██████████| 30490/30490 [00:01<00:00, 18301.68it/s]


In [16]:
for d in tqdm(range(1, 29)):
    sales_train_validation['f_1_'+str(1885+d)] = list(complete_historical_mean)
    sales_train_validation['f_2_'+str(1885+d)] = non_zero_historical_mean

method_dict[1] = "complete historical mean"
method_dict[2] = "non zero historical mean"

100%|██████████| 28/28 [00:00<00:00, 78.17it/s]


### Mean of # recent days (10, 20, 30, 40, 50)

In [17]:
historical_mean_10_days = sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_') == 0 and int(c.split('_')[1]) in range(1876, 1886)]].mean(axis=1).round().to_list()
historical_mean_20_days = sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_') == 0 and int(c.split('_')[1]) in range(1866, 1886)]].mean(axis=1).round().to_list()
historical_mean_30_days = sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_') == 0 and int(c.split('_')[1]) in range(1856, 1886)]].mean(axis=1).round().to_list()
historical_mean_40_days = sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_') == 0 and int(c.split('_')[1]) in range(1846, 1886)]].mean(axis=1).round().to_list()
historical_mean_50_days = sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_') == 0 and int(c.split('_')[1]) in range(1836, 1886)]].mean(axis=1).round().to_list()

In [18]:
for d in tqdm(range(1, 29)):
    sales_train_validation['f_3_' + str(1885+d)] = historical_mean_10_days
    sales_train_validation['f_4_' + str(1885+d)] = historical_mean_20_days
    sales_train_validation['f_5_' + str(1885+d)] = historical_mean_30_days
    sales_train_validation['f_6_' + str(1885+d)] = historical_mean_40_days
    sales_train_validation['f_7_' + str(1885+d)] = historical_mean_50_days

method_dict[3] = "historical mean of last 10 days"
method_dict[4] = "historical mean of last 20 days"
method_dict[5] = "historical mean of last 30 days"
method_dict[6] = "historical mean of last 40 days"
method_dict[7] = "historical mean of last 50 days"

 50%|█████     | 14/28 [00:00<00:00, 63.50it/s]C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\2308148260.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_train_validation['f_5_' + str(1885+d)] = historical_mean_30_days
C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\2308148260.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_train_validation['f_6_' + str(1885+d)] = historical_mean_40_days
C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\2308148260.py:6: PerformanceWarning: DataFrame is highly 

### Same as previous 28 days

In [19]:
for d in tqdm(range(1, 29)):
    sales_train_validation['f_8_'+ str(1885+d)] = sales_train_validation['d_' + str(1885 + d - 28)].to_list()
method_dict[8] = "same as previous 28 days"

  0%|          | 0/28 [00:00<?, ?it/s]C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\3233770312.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_train_validation['f_8_'+ str(1885+d)] = sales_train_validation['d_' + str(1885 + d - 28)].to_list()
C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\3233770312.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_train_validation['f_8_'+ str(1885+d)] = sales_train_validation['d_' + str(1885 + d - 28)].to_list()
C:\Users\923006079\AppData\Local\Temp\ipykernel_13

## 3. Infer forecast, ground truth values, and weights for all the higher level series by aggregating

In [20]:
agg_df = pd.DataFrame(sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_')==0 or c.find('f_')==0]].sum()).transpose() # .transpose()
agg_df['level'] = 1
agg_df['weight'] = 1/12
column_order = agg_df.columns

In [21]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,f_8_1906,f_8_1907,f_8_1908,f_8_1909,f_8_1910,f_8_1911,f_8_1912,f_8_1913,level,weight
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,47825.0,37360.0,35475.0,34786.0,34003.0,45611.0,53863.0,46360.0,1,0.083333


In [22]:
level_groupings = {
    2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
    6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 
    9: ["store_id", "dept_id"], 10: ["item_id"], 11: ["item_id", "state_id"] # , 12: ["item_id", "store_id"]
}

In [23]:
for level in tqdm(level_groupings):
    temp_df = sales_train_validation.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
    temp_df['level'] = level
    temp_df['weight'] /= 12
    agg_df = pd.concat([agg_df, temp_df[column_order]])

del temp_df

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\146794995.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp_df = sales_train_validation.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
 10%|█         | 1/10 [00:01<00:09,  1.03s/it]C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\146794995.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp_df = sales_train_validation.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
 20%|██        | 2/10 [00:01<00:06,  1.17it/s]C:\Users\923006079\AppData\Local\Temp\ipykernel_13076\146794

In [24]:
sales_train_validation['weight'] /= 12

In [25]:
print(sales_train_validation.shape[0], agg_df.shape[0], sales_train_validation.shape[0] + agg_df.shape[0])

30490 12350 42840


In [26]:
print(sales_train_validation['weight'].sum(), agg_df['weight'].sum(), sales_train_validation['weight'].sum() + agg_df['weight'].sum())

0.08333333333333334 0.9166666666666666 1.0


## 4. Calculate RMSSE for All The Series

In [27]:
train_series_cols = [c for c in sales_train_validation.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in sales_train_validation.columns if c.find("d_") == 0][-28:]

In [28]:
h = 28
n = 1885
# rmsse need 4 data - horizon, ts for training, ts for forecast, ts, for 
def rmsse(ground_truth, forecast, train_series, axis=1):
    assert axis==0 or axis==1
    assert type(ground_truth) == np.ndarray or type(forecast) == np.ndarray or type(train_series) == np.ndarray
    
    if axis  == 1:
        # if axis = 1 we have to make sure that data are in matrix format
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis==1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1])**2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1])**2).sum(axis=axis)
    
    return (1/h * numerator / denominator) ** 0.5

In [29]:
for k, v in method_dict.items():
    sales_train_validation['rmsse'] = rmsse(
        np.array(sales_train_validation[ground_truth_cols]),
        np.array(sales_train_validation[[c for c in sales_train_validation.columns if c.find('f_'+str(k)) == 0]]),
        np.array(sales_train_validation[train_series_cols])
    )
    agg_df["rmsse"] = rmsse(
        np.array(agg_df[ground_truth_cols]), 
        np.array(agg_df[[c for c in sales_train_validation.columns if c.find('f_'+str(k)) == 0]]), 
        np.array(agg_df[train_series_cols])
    )
    sales_train_validation['wrmsse'] = sales_train_validation['weight'] * sales_train_validation['rmsse']
    agg_df['wrmsse'] = agg_df['weight'] * agg_df['rmsse']
    print(v)
    print("WRMSSE", sales_train_validation['wrmsse'].sum() + agg_df['wrmsse'].sum(), "\n")


using all 0s
WRMSSE 5.359792730956583 

complete historical mean
WRMSSE 1.654145319495255 

non zero historical mean
WRMSSE 4.150683531291453 

historical mean of last 10 days
WRMSSE 1.23521237018701 

historical mean of last 20 days
WRMSSE 1.2220278819228305 

historical mean of last 30 days
WRMSSE 1.1912043925125957 

historical mean of last 40 days
WRMSSE 1.205913226789875 

historical mean of last 50 days
WRMSSE 1.2018883868953785 

same as previous 28 days
WRMSSE 0.8558233199674038 

